In [14]:
%reload_ext autoreload
%autoreload 2
import os
import ee
from IPython.display import Image
import datetime
from matplotlib import dates
import matplotlib.dates as mdates
from pylab import *
from pyLib.spectral_indices import *
import numpy as np
import json
import pandas as pd
from pandas.io.json import json_normalize
from pyLib.gee_utility import *
from pyLib.const import *

try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    

        
def addAll(image):
    image = image.addBands(GEMI(image))
    image = image.addBands(NDVI(image))
    image = image.addBands(NDMI(image))
    return image
    
def forest_detection(polygon_shp, start_year, end_year, scale=30):
    region_name = os.path.basename(polygon_shp).rsplit('.', 1)[0]
    id_name = region_name
    espg_code = 'EPSG:4326'
    min_area = 100000 #  10 ha
    min_area = 50000 #  1 ha
    min_size = max(1, round(min_area/float(scale**2)))
    print('minimum connected pixels: %d' %(min_size))
    
    dataset_name_list = ['LANDSAT/LC08/C01/T1_TOA', 'LANDSAT/LC08/C01/T2_TOA']
#     dataset_name_list = ['LANDSAT/LT05/C01/T1_TOA', 'LANDSAT/LT05/C01/T2_TOA']
    gfc = ee.Image('UMD/hansen/global_forest_change_2015')
    
    bound_feature = read_polygon_collection(polygon_shp)    
    sample_feature = ee.FeatureCollection.randomPoints(bound_feature, 3000)
    
    collection = get_landsat_toa_collection(dataset_name_list).filterBounds(bound_feature)\
                    .filterDate("{}-1-1".format(start_year),\
                                "{}-12-30".format(end_year))
    collection = collection.filter(ee.Filter.calendarRange(7, 12, 'month'))
    collection = collection.map(maskCloud)
    img = collection.median()
    img = img.addBands(gfc.select('treecover2000').gt(20)\
                       .And(gfc.select('loss').Not()).Or(gfc.select('gain')).rename('CLASS'))
    img = img.addBands(GEMI(img))
    img = img.addBands(NDVI(img))
    
#     band_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'NDVI', 'GEMI', 'CLASS']
    feature_names = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'NDVI', 'GEMI']
    band_list = feature_names + ['CLASS']
    samples = img.sampleRegions(collection=sample_feature, scale=30)
    samples = samples.select(band_list)
    
    clf_model = 'CLASSIFICATION'
#     clf_model = 'PROBABILITY'
    clf = ee.Classifier.randomForest(numberOfTrees=50)\
        .train(samples, classProperty='CLASS', inputProperties=feature_names)\
        .setOutputMode(clf_model)

    if clf_model == 'CLASSIFICATION':
        trainAccuracy = clf.confusionMatrix()
        print('Resubstitution error matrix: ', trainAccuracy.getInfo())
        print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())
#         return    
        img = img.addBands(img.select(feature_names).classify(clf, outputName='CLASS').rename('CLASS'))
        forest = img.select('CLASS')
    else:    
        img = img.addBands(img.select(feature_names).classify(clf, outputName='PROBABILITY').rename('PROBABILITY'))    
        forest = img.select('PROBABILITY').gte(0.85)
    
    forest = forest.mask(forest.connectedPixelCount(100, True).gt(min_size)).clip(bound_feature)
    kernel = ee.Kernel.square(5)
    forest = forest.reduceNeighborhood(ee.Reducer.median(), kernel)

    task= ee.batch.Export.image.toDrive(
        crs= espg_code,
        image=forest,
        description=id_name,
        folder= "forest_{}-{}m-learn".format(start_year, scale),
        fileNamePrefix=id_name,
        region= bound_feature.getInfo()['features'][0]['geometry']["coordinates"],
        scale= scale,
        maxPixels= 1e13,
        shardSize= 512,
        fileDimensions=65536,
#             fileDimensions=131072,
        skipEmptyTiles=True)
    task.start()
    
    
polygon_shp = r'data/SouthAsia.shp'
forest_detection(polygon_shp, 2015, 2015, scale=30)

The Earth Engine package initialized successfully!
minimum connected pixels: 56
